# Финальный проект Python

In [ ]:
mport matplotlib.pyplot as plt
import seaborn as sns
import re as re
import pandas as pd
import math
df=pd.read_csv(r'C:\csv\Финальный_проект_данные.csv')

# ОЦЕНИТЕ ОБЪЕМ РЫНКА
# Отфильтруйте датасет и выведите данные только о машинах, произведенных с 2017 года.Данные в 'full_name'
df['year']=df['full_name'].apply(lambda x: x[:4])
df['year']=df['year'].astype('int')

# Добавьте еще один фильтр и посмотрите, какая доля автомобилей в получившемся датасете имеет объем двигателя менее 2000 сс. 
# Чтобы «извлечь» из поля «engine_capacity» только числовые символы, используйте регулярное 
# выражение apply(lambda x: (re.findall(r'\d+', x))[0]). Мощность может включать как 3, так и 4 цифры, 
# поэтому в этом случае код будет немного сложнее, чем в предыдущем.
df['engine_cap']=df['engine_capacity'].apply(lambda x: (re.findall(r'\d+', x))[0])
df['engine_cap']=df['engine_cap'].astype('int')

# Отфильтруйте данные:
# — посчитайте, сколько строк содержат информацию о продаже автомобилей, произведенных начиная с 2017 года;
# — посчитайте, сколько строк содержат информацию о продаже автомобилей, произведенных начиная с 2017 года и 
# имеющих объём двигателя до 2000 с помощью  'shape'
# Разделите второе число на первое. Округлите ответ до двух знаков после запятой.
print('кол-во машин произведенных с 2017 всего:', len(df.query('year >=2017')),df.query('year >=2017').shape[0])
print('кол-во среди них машин с двигателем менее 2000 куб.см. :', len(df.query('year>=2017 and engine_cap<2000')),df.query('year>=2017 and engine_cap<2000').shape[0])
print('доля машин с двигателем менее 2000 куб.см. среди тех, что произведены с 2017г.:', round(len(df.query('year>=2017 and engine_cap<2000'))/len(df.query('year>=2017')),2))        

# df = df.reset_index(drop=1)                 # Сброс индексов, чтобы в нашем датасете не было пропусков. Аргумент drop=1 означает, 
                                             # что нам не нужно сохранять в таблице значения старых индексов, потому что 
                                             # мы хотим заменить их новыми.
        

# УСТАНОВИТЕ ЗАВИСИМОСТЬ СРЕДНЕЙ ЦЕНЫ ПРОДАЖИ ОТ ГОДА ВЫПУСКА АВТОМОБИЛЯ
# Переведите Рупии (из 'Resale price') в рубли по курсу 1 рупия = 1.16 рубля.
price_=[]
df['resale_price']=df['resale_price'].apply(lambda x: x.split(' '))
for i in range(len(df['resale_price'])):
    if df['resale_price'][i][2]==str("Lakh_"):
        price_.append(float(df['resale_price'][i][1])*100000*1.16)
    elif df['resale_price'][i][2]=='Crore':
        price_.append(float(df['resale_price'][i][1])*10000000*1.16)
df['RUB']=price_ 

# Посчитайте среднюю стоимость проданных автомобилей по году выпуска
# Воспользуйтесь группировкой, чтобы посчитать среднее за каждый год.

df_2017=df.query('year>=2017')

df_gr=df_2017.groupby('year')['RUB'].agg(['mean']). round(2)    # добавили [] к 'mean' чтобы получить не список группировок, а Pandas таблицу

# Постройте график, который будет отображать зависимость цены от года выпуска автомобиля, 
# присвоив значение года выпуска оси Х, а значение средней стоимости — оси Y. Нас интересуют 
# только автомобилями, произведенные с 2017 года.
df_gr = df_gr.reset_index()
sns.barplot(data=df_gr.query('year>=2017'), x='year',y='mean')

# Создали столбец 'km' пробега (int)
km=[]
df_2017['kms_driven']=df_2017['kms_driven'].apply(lambda x: x.split(' '))
for i in range(len(df_2017['kms_driven'])):                             # Создали столбец пробега без текста 'int'
    km.append((df_2017['kms_driven'][i][0]))
df_2017['km']=km 
df_2017['km']=df_2017['km'].apply(lambda x: x.replace(',',''))
df_2017['km']=df_2017['km'].astype('float')

# выполните проверку: определите, какое количество автомобилей имеет пробег, превышающий 200 000 километров. 
# Если таких автомобилей окажется менее 1% от общего объема рынка, исключите их при построении гистограммы.
print('кол-во машин всего:', len(df_2017['km']))
print('кол-во машин с пробегом более 200000 км:', len(df_2017.query('km>200000')))
print('доля машин с пробегом более 200000 км:', len(df_2017.query('km>200000'))/len(df_2017['km']))

# sns.histplot(data=df.query('km<=200000'), x='km', binwidth=10000) # ХЗ ПОЧЕМУ НЕ РАБОТАЕТ????!!!!

# ПОСЧИТАЙТЕ, СКОЛЬКО НОВЫХ МОДЕЛЕЙ ПОЯВИЛОСЬ НА РЫНКЕ С 2022 ПО 2023Г.
# выделите все модели, которые продавались в 2022 и 2023 году;
# выделите все модели, которые продавались до 2021 года включительно.
df_2017['full_name']=df_2017['full_name'].apply(lambda x: x[5:])   # Убрали первые 5 символов


# Создайте два множества: в одно добавьте названия моделей, которые продавались в 2022 и 2023 году, 
# во второе — названия моделей, которые продавались до 2021 года включительно. 
# set(df.query('условие')['столбец, значения которого кладем в множество']).
models_22_23=[]
models_up_21=[]
new_models=[]
models_22_23=set(df_2017.query('year==2022 or year==2023')['full_name'])
models_up_21=set(df_2017.query('year<2021')['full_name'])
new_models=models_22_23.difference(models_up_21)                  #оставили значения, которые есть в 22-23гг и нет в до 21г вкл.
print('Количество новых моделей с 2022г.:', len(new_models))

# Создали столбец 'max_power' с цифрами без букв 'float'
df_2017['max_power']=df_2017['max_power'].apply(lambda x: re.findall(r'\d+', x))
df_2017['max_power']=df_2017['max_power'].apply(lambda x: float('.'.join(x)))
df_2017['max_power']=df_2017['max_power'].astype('float')                     #сказали сделать 'str' чтоб не было ошибки. ХЗ почему?

# df_2017=df.query('year>=2017')
df_2017 = df_2017.reset_index()

yyy=[]

def index_(price, power, kms):
    ind=(math.log(price,2)*math.sqrt(power))/math.log((kms/100),2)
    return(ind)

for y in range(len(df_2017)):
    yyy.append((index_(df_2017['RUB'][y], df_2017['max_power'][y], df_2017['km'][y])))
    
df_2017['index']=yyy

display(df_2017)